# Blurred Ball Cover

Data Source: https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones

# Import Modules and Packages

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
from collections import namedtuple
import numpy.linalg as npl
from sets import Set

/Library/Python/2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: the sets module is deprecated
  


# Declare Data Structures

In [2]:
Ball = namedtuple("Ball", ["center", "radius"])
BlurredBall = namedtuple("BlurredBall", ["k", "MEB"])

# Methods to Generate and Read Data

In [44]:
def generate_normal_data(num_of_points, num_of_dimensions):
    mean = random.sample(np.array(range(100)), num_of_dimensions)
    scalar_variances = 10.0 * np.ones(num_of_dimensions)
    covariance = np.diag(scalar_variances)
    points = np.random.multivariate_normal(mean, covariance, num_of_points)
    return points

def generate_multivariate_data(num_of_points, num_of_dimensions):
    probabilities = [float(1.0 / num_of_dimensions)] * num_of_dimensions
    points = np.random.multinomial(20, probabilities, size = num_of_points)
    return points

def read_data(data_file):
    data = []
    with open(data_file) as data_reader:
        lines = data_reader.readlines()
        for line in lines:
            values = line.split()
            #values = values[1:]
            values = values[:400]
            row = [float(value) for value in values]
            data.append(row)
        data = np.array(data)
        print(data.shape)
        return data, data.shape[0], data.shape[1]


# Helper Functions for Computing Clarkson's (2/epsilon)-coreset and MEB 

In [45]:
def inside_ball_eps(blurred_ball, point, epsilon):
    #print "inside_ball"
    center, radius = blurred_ball.MEB
    distance = npl.norm(point - center)
    if distance <= (1 + epsilon) * radius:
        return True
    else:
        return False

In [46]:
def approx_meb_light(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0
    num_iterations = int(1.0 / (epsilon * epsilon))
    distances = {}
    prev_radius = -10000.0
    coreset = [points[0]]
    for i in range(1, num_iterations):
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]

        center = center + (1.0 / i) * (furthest_point - center)   
        if abs(furthest_distance - prev_radius) < 0.01:
            break
        prev_radius = furthest_distance
            
    radius = npl.norm(furthest_point - center)
    meb = Ball(center, radius)
    return meb

In [47]:
def epsilon_coreset(points, epsilon):
    coreset = []
    num_of_points = len(points)
    furthest_point = None
    center = points[0]
    radius = 0.0

    num_iterations = int(2.0 / (epsilon))
    print("Total Iterations: " + repr(num_iterations))
    distances = {}
    coreset_indices = Set()

    prev_radius = -10000.0
    
    coreset = [points[0]]
    for i in range(1, num_iterations):
        #print("Iteration: ", i, " radius: ", prev_radius)
        # Find furthest point
        for j in range(num_of_points):
            distances[j] = npl.norm(points[j] - center)
        # Calculated furthest point    
        furthest_point_index = max(distances, key = distances.get)
        furthest_point = points[furthest_point_index]
        furthest_distance = distances[furthest_point_index]
        if furthest_point_index in coreset_indices:
            continue
        coreset_indices.add(furthest_point_index)
        coreset.append(furthest_point)
        coreset_meb = approx_meb_light(coreset, 0.01)
        center = coreset_meb.center
        #print center, coreset_meb.radius
        
    return coreset

# Clarkson's 2/epsilon coreset and MEB Algorithm

In [48]:
def approx_meb(points, epsilon):
    coreset = epsilon_coreset(points, epsilon)
    meb = approx_meb_light(coreset, epsilon)
    return BlurredBall(coreset, meb)
    

# Blurred Ball Cover Algorithm's Helper Method

In [49]:
def update(blurred_balls, A, epsilon):
    print "update"
    K = []
    for blurred_ball in blurred_balls:
        K.extend(blurred_ball.k)


    outer_loop_flag = False    
    for point in A:
        if outer_loop_flag == True:
            break
        for blurred_ball in blurred_balls:
            if inside_ball_eps(blurred_ball, point, epsilon) == False:
                outer_loop_flag = True
                break
                
    if outer_loop_flag == False:
        #print("This point is covered")
        return blurred_balls
    #else:
        #print("This point is not covered")
        
    K_union_A = list(K)
    K_union_A.extend(A)
    blurred_ball_new = approx_meb(K_union_A, epsilon / 3.0)
    
    discardables = []
    for blurred_ball in blurred_balls:
        lhs = blurred_ball.MEB.radius
        rhs = epsilon * blurred_ball_new.MEB.radius / 4.0
        #print("lhs <> rhs: " + repr(lhs) + " <> " + repr(rhs))
        if blurred_ball.MEB.radius <= (epsilon * blurred_ball_new.MEB.radius / 4.0):
            print "Found a discardable"
            discardables.append(blurred_ball)
    
    blurred_balls = [bb for bb in blurred_balls if bb not in np.array(discardables)]
    
    blurred_balls.append(blurred_ball_new)
    
    return blurred_balls
        

# Blurred Ball Cover Algorithm

In [50]:
def agarwal(points, num_of_points, num_of_dimensions):
    epsilon = 0.1
    batch_size = 100
    blurred_balls = []
    initial_points = points[0:num_of_dimensions, :]
    blurred_ball_init = approx_meb(initial_points, epsilon / 3.0)
    blurred_balls = [blurred_ball_init]

    for i in range(num_of_dimensions, num_of_points, batch_size):
        print "Iteration: " + repr(i)
        sid = i
        fid = min(i + batch_size, num_of_points)
        A = points[sid : fid, :]
        blurred_balls = update(blurred_balls, A, epsilon)
        
    return blurred_balls
  

# Drawing Utility Methods : For Debug and Visualization Purpose

In [51]:
def draw_blurred_balls(blurred_balls):
    xmin = min(points[:, 0])
    xmax = max(points[:, 0])
    ymin = min(points[:, 1])
    ymax = max(points[:, 1])

    print(xmin, xmax, ymin, ymax)
    # ball <- \phi
    ball = None
    count = 0
    processed_points = []
    fig, ax = plt.subplots(1, 1) 
    #ax.set_xlim(xmin - 10, xmax + 10)
    #ax.set_ylim(ymin - 10, ymax + 10)
    ax.set_aspect('equal', 'datalim')
    ax.set_xlim(xmin - 10, xmax + 10)
    ax.set_ylim(ymin - 10, ymax + 10)


    X = points[:, 0]
    Y = points[:, 1]
    scatters = plt.scatter(X, Y)

    for blurred_ball in blurred_balls:
        #print blurred_ball
        center, radius = blurred_ball.MEB
        circle = plt.Circle(center, radius, color='r', alpha = 0.1)
        ax.add_artist(circle)
        ax.add_artist(scatters)
    plt.show()

In [52]:
def draw_points_and_meb(points, meb):
    xmin = min(points[:, 0])
    xmax = max(points[:, 0])
    ymin = min(points[:, 1])
    ymax = max(points[:, 1])
    print(xmin, xmax, ymin, ymax)
    processed_points = []
    fig, ax = plt.subplots(1, 1) 
    #ax.set_xlim(xmin - 10, xmax + 10)
    #ax.set_ylim(ymin - 10, ymax + 10)
    ax.set_aspect('equal', 'datalim')
    ax.set_xlim(xmin - 50, xmax + 50)
    ax.set_ylim(ymin - 50, ymax + 50)
    X = points[:, 0]
    Y = points[:, 1]
    scatters = plt.scatter(X, Y)
    center = meb.center
    radius = meb.radius
    circle = plt.Circle(center, radius, color='r', alpha = 0.1)
    ax.add_artist(circle)

# Read Data from File

In [53]:
points, num_of_points, num_of_dimensions = read_data("human_activity.txt")
print num_of_points, num_of_dimensions

(7352, 400)
7352 400


# Run Blurred Ball Cover Algorithm

In [54]:
blurred_balls = agarwal(points, num_of_points, num_of_dimensions)
#draw_blurred_balls(blurred_balls)

Total Iterations: 60
Iteration: 400
update
Total Iterations: 60
Iteration: 500
update
Iteration: 600
update
Iteration: 700
update
Iteration: 800
update
Total Iterations: 60
Iteration: 900
update
Total Iterations: 60


/Library/Python/2.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Iteration: 1000
update
Iteration: 1100
update
Total Iterations: 60
Iteration: 1200
update
Total Iterations: 60
Iteration: 1300
update
Total Iterations: 60
Iteration: 1400
update
Total Iterations: 60
Iteration: 1500
update
Total Iterations: 60
Iteration: 1600
update
Total Iterations: 60
Iteration: 1700
update
Total Iterations: 60
Iteration: 1800
update
Total Iterations: 60
Iteration: 1900
update
Total Iterations: 60
Iteration: 2000
update
Total Iterations: 60
Iteration: 2100
update
Total Iterations: 60
Iteration: 2200
update
Total Iterations: 60
Iteration: 2300
update
Total Iterations: 60
Iteration: 2400
update
Total Iterations: 60
Iteration: 2500
update
Total Iterations: 60
Iteration: 2600
update
Total Iterations: 60
Iteration: 2700
update
Total Iterations: 60
Iteration: 2800
update
Total Iterations: 60
Iteration: 2900
update
Total Iterations: 60
Iteration: 3000
update
Total Iterations: 60
Iteration: 3100
update
Total Iterations: 60
Iteration: 3200
update
Iteration: 3300
update
Total I

# Inspect Blurred Ball Cover's Output

In [55]:
for i in range(len(blurred_balls)):
    print i, len(blurred_balls[i].k), blurred_balls[i].MEB.radius

0 7 7.66843560742
1 7 8.49862174686
2 7 8.49862174686
3 6 8.48599154945
4 6 8.47995810854
5 4 9.51785172847
6 3 9.65518042755
7 3 9.65518042755
8 6 9.42774122735
9 6 9.42774122735
10 6 9.81110169698
11 5 9.85057075957
12 5 9.84161235963
13 5 9.84161235963
14 5 9.84161235963
15 5 9.84161235963
16 5 9.84161235963
17 6 10.0271296905
18 8 10.2927722037
19 10 10.2861243014
20 10 10.2861243014
21 10 10.2861243014
22 10 10.2861243014
23 10 10.2861243014
24 10 10.2861243014
25 10 10.2861243014
26 10 10.2861243014
27 8 10.3030411569
28 8 10.3030411569
29 3 10.017641932
30 3 10.017641932
31 7 12.3003271002
32 7 12.3003271002
33 7 12.3003271002
34 7 12.3003271002
35 7 12.3003271002
36 7 12.3003271002
37 7 12.3003271002
38 7 12.3003271002
39 5 12.2550837654
40 5 12.2550837654
41 5 12.2550837654
42 5 12.2550837654
43 3 12.3280136368
44 3 12.3280136368
45 3 12.3280136368
46 3 12.3280136368
47 3 12.3280136368
48 3 12.3280136368
49 3 12.3280136368
50 3 12.3280136368
51 3 12.3280136368
52 3 12.32801363

# Write the Blurred Balls into a File

In [56]:
N = len(blurred_balls)
D = num_of_dimensions

with open("activity_balls.txt", "w") as outFile:
    outFile.write(repr(N) + " " + repr(D) + "\n")
    for ball in blurred_balls:
        print ball.MEB.center, ball.MEB.radius
        line = ""
        for d in range(D):
            line += repr(ball.MEB.center[d]) + " "
        line += repr(ball.MEB.radius) + "\n"
        outFile.write(line)
        #print line

[ 0.28257    -0.02039606 -0.16367259 -0.50999331 -0.42742997 -0.55843271
 -0.54888283 -0.47867423 -0.55530985 -0.32057529 -0.30574223 -0.55592072
  0.42406449  0.2272962   0.54110795 -0.46455521 -0.74017597 -0.86480825
 -0.83116264 -0.63998399 -0.66685795 -0.54621464 -0.25200539 -0.20545245
 -0.34680054 -0.02182747  0.02025046 -0.07254538  0.19105205  0.09954495
 -0.07424944  0.20200021 -0.03366771  0.18312228 -0.09475149  0.16309906
 -0.14805206 -0.04529063  0.11534995  0.24844467  0.349378    0.15405113
  0.10234522 -0.9602218  -0.9314286  -0.85125547 -0.9630865  -0.93130171
 -0.85536587  0.29354072  0.13632467  0.11909322  0.36778264  0.15349088
  0.05847824 -0.16276595  0.03046206 -0.42077373 -0.66720827 -0.97167094
 -0.93237423 -0.86761255 -0.48226843 -0.81845751 -0.77679277 -0.6093039
  0.64338795 -0.67724107  0.71094921 -0.42351231  0.41285122 -0.44142658
  0.4859437  -0.43101696  0.4483361  -0.46511604  0.47876791  0.05933674
  0.0721687   0.3070991   0.11448129 -0.03900341 -0.

[  2.73147250e-01  -3.49004199e-02  -1.35321435e-01  -3.59006733e-01
  -1.76688931e-01  -4.21787160e-01  -3.96678041e-01  -2.58644986e-01
  -4.45307478e-01  -2.33888503e-01  -1.54315527e-01  -3.91104544e-01
   3.39456834e-01  -2.49393404e-02   4.44045372e-01  -3.01634350e-01
  -5.79300019e-01  -7.33520560e-01  -7.02392593e-01  -4.95690843e-01
  -5.48818517e-01  -5.55769713e-01  -1.25715849e-01  -2.25835416e-01
  -1.58262093e-01   1.79905493e-02  -3.20383713e-02  -4.13972522e-02
   2.12095280e-01   5.87908987e-02  -4.14118870e-03   1.78089744e-01
  -1.13093343e-02   2.32862454e-01  -3.88233057e-02   8.42468913e-02
   6.11694617e-02  -5.45366487e-02  -2.57426874e-01  -5.75977285e-02
   4.47299547e-01   1.03689726e-01   2.55848043e-01  -9.46311209e-01
  -9.36138067e-01  -9.14566152e-01  -9.45812878e-01  -9.34201490e-01
  -9.15229616e-01   3.94017724e-01   9.08704800e-02   2.62092391e-01
   4.61793009e-01   1.13885438e-01   2.36392588e-01  -3.21205380e-01
   1.23399234e-01  -5.95137858e-01

[  2.40368641e-01  -1.65770279e-02  -1.28694821e-01  -3.81740052e-01
  -1.43671444e-01  -1.44703124e-01  -4.52958798e-01  -2.50066622e-01
  -1.93833853e-01  -1.73686212e-01  -6.40776494e-02  -3.56353237e-01
   3.27763377e-01  -1.86559082e-02   1.68807716e-01  -2.57414053e-01
  -6.26612805e-01  -7.26961942e-01  -3.50631866e-01  -6.02084184e-01
  -5.42411224e-01  -3.54180466e-01  -1.33707572e-01  -1.73879626e-01
  -7.13288319e-02   1.58799000e-02  -9.52153352e-02   1.65037275e-01
   5.48158126e-02   9.53690095e-02  -6.39690222e-02   2.66155256e-01
  -5.58087391e-02   1.10962223e-01  -9.64354104e-03  -3.12426323e-02
   1.49458347e-01  -1.67467956e-01  -3.48501370e-01   7.03998713e-02
   3.79745266e-01   1.29325748e-01   1.05121926e-01  -9.11727688e-01
  -9.30169379e-01  -8.85587634e-01  -9.11946098e-01  -9.28482470e-01
  -8.86557730e-01   3.34829149e-01   1.14799967e-01   1.17674707e-01
   3.81529846e-01   1.35193702e-01   7.63654212e-02  -1.74603708e-02
  -2.27044495e-02  -5.47978050e-01

[ 0.14818794 -0.01964318 -0.11063558 -0.08675247 -0.03806143 -0.13093265
 -0.10147281 -0.06166952 -0.15478787 -0.1132195  -0.22959543 -0.24774193
  0.08306636  0.05095999  0.07954316  0.01637136 -0.17516181 -0.65265812
 -0.36215119 -0.13928509 -0.30345087 -0.24713148 -0.19777596 -0.12331726
 -0.04698785 -0.06198052  0.0300183  -0.05908049  0.24299191  0.14091626
 -0.06963425  0.14821826  0.14191028  0.22977305  0.00829097 -0.05659036
  0.14753881 -0.16747523 -0.26746142  0.24270491  0.45550288  0.14978625
  0.07789014 -0.87142264 -0.94710711 -0.93233953 -0.8677935  -0.94637115
 -0.9383532   0.42011095  0.13260193  0.08181912  0.44352492  0.15875799
  0.05820237 -0.14266858  0.08442644 -0.51654005 -0.68750186 -0.85683985
 -0.94536774 -0.95130616 -0.10747132 -0.8575719  -0.75504548 -0.63563947
  0.69508091 -0.75534098  0.8164808  -0.36863173  0.38622217 -0.44848219
  0.5275354  -0.15318709  0.21457659 -0.27580691  0.33443332  0.24700671
 -0.05510218  0.31206407 -0.07888123  0.23236498  0

[  1.58543202e-01  -1.62189775e-02  -1.14961693e-01  -1.00928330e-01
  -3.34363250e-02  -9.16965550e-02  -1.16830360e-01  -5.86140750e-02
  -1.21061375e-01  -1.25643195e-01  -2.39582414e-01  -2.14396695e-01
   8.66969900e-02   3.98595700e-02   5.12335550e-02   1.72386050e-02
  -1.91396510e-01  -6.48123545e-01  -3.35671785e-01  -1.57984195e-01
  -3.10758355e-01  -2.28593495e-01  -8.76952150e-02  -7.66821700e-02
  -1.01834895e-01   1.97498585e-01  -1.79044585e-01  -3.65253000e-03
   1.73222509e-01   1.01237547e-01  -8.36594545e-02   1.92975365e-01
   6.18298600e-02   2.71920215e-01  -1.51835665e-01   1.83738240e-01
  -1.93252550e-02  -1.22905190e-01  -4.72835150e-02   6.10541695e-01
   9.15817590e-01  -1.72154735e-01  -3.80041450e-02  -8.74671365e-01
  -9.50311480e-01  -9.07360635e-01  -8.71070665e-01  -9.50787690e-01
  -9.19726920e-01   8.75666920e-01  -1.82276765e-01  -3.06315300e-02
   8.93322685e-01  -1.60470050e-01  -7.56757180e-02  -2.65306085e-01
   7.79190690e-01  -9.52073730e-01

  -3.23432850e-01  -6.01853510e-01  -7.10857310e-01  -5.45630360e-01] 12.3280136368
[  1.58543202e-01  -1.62189775e-02  -1.14961693e-01  -1.00928330e-01
  -3.34363250e-02  -9.16965550e-02  -1.16830360e-01  -5.86140750e-02
  -1.21061375e-01  -1.25643195e-01  -2.39582414e-01  -2.14396695e-01
   8.66969900e-02   3.98595700e-02   5.12335550e-02   1.72386050e-02
  -1.91396510e-01  -6.48123545e-01  -3.35671785e-01  -1.57984195e-01
  -3.10758355e-01  -2.28593495e-01  -8.76952150e-02  -7.66821700e-02
  -1.01834895e-01   1.97498585e-01  -1.79044585e-01  -3.65253000e-03
   1.73222509e-01   1.01237547e-01  -8.36594545e-02   1.92975365e-01
   6.18298600e-02   2.71920215e-01  -1.51835665e-01   1.83738240e-01
  -1.93252550e-02  -1.22905190e-01  -4.72835150e-02   6.10541695e-01
   9.15817590e-01  -1.72154735e-01  -3.80041450e-02  -8.74671365e-01
  -9.50311480e-01  -9.07360635e-01  -8.71070665e-01  -9.50787690e-01
  -9.19726920e-01   8.75666920e-01  -1.82276765e-01  -3.06315300e-02
   8.93322685e-01  

In [57]:
import numpy as np
from sklearn.decomposition import PCA

def get_pca_transformer(points_hd):
    pca = PCA(n_components=2)
    pca.fit(points_hd)
    return pca

def get_blurred_ball_2d(blurred_ball, pca):
    points_hd = blurred_ball.k
    center_hd = blurred_ball.MEB.center
    points_hd.append(center_hd)
    
    points_2d = pca.fit_transform(points_hd)
    num_of_points = len(points_2d)
    center_2d = points_2d[num_of_points - 1, :]
    radius = blurred_ball.MEB.radius
    points_2d = points_2d[:num_of_points - 2, :]
    blurred_ball_2d = BlurredBall(points_2d, Ball(center_2d, radius))
    return blurred_ball_2d

In [58]:
'''
pca = get_pca_transformer(points)
blurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]
print(blurred_balls_2d[0])
draw_blurred_balls(blurred_balls_2d)
'''

'\npca = get_pca_transformer(points)\nblurred_balls_2d = [get_blurred_ball_2d(blurred_ball, pca) for blurred_ball in blurred_balls]\nprint(blurred_balls_2d[0])\ndraw_blurred_balls(blurred_balls_2d)\n'

# Timothy Chan's Algorithm

In [59]:
Ball = namedtuple("Ball", ["center", "radius"])

def inside_ball(ball, point):
    center, radius = ball
    distance = npl.norm(point - center)
    if distance < radius:
        return True
    else:
        return False

def meb_ball_and_point(ball, p):
    c, r = ball
    pc_scalar = npl.norm(c - p)
    pc_vector = c - p
    radius_unit = pc_vector / pc_scalar
    p_mirror = radius_unit * r + c
    c_prime = (p + p_mirror) / 2.0
    r_prime = npl.norm(p_mirror - p) / 2.0
    meb = Ball(c_prime, r_prime)
    return meb

def create_initial_ball(point):
    return Ball(point, 0.0)

def chan(points):
    count = 0
    center = None
    radius = None
    ball = None
    for point in points:
        if ball == None:
            ball = create_initial_ball(point)
            continue
        if inside_ball(ball, point):
            continue
        else:
            count += 1
            ball = meb_ball_and_point(ball, point)
            center, radius = ball
    return Ball(center, radius)

In [61]:
meb = chan(points)
print(meb)
print meb.center.shape
    

Ball(center=array([ 0.1796176 , -0.01893758, -0.063047  , -0.32146503, -0.09287586,
       -0.29403404, -0.35357894, -0.12204279, -0.30941012, -0.26994147,
       -0.14032989, -0.2858553 ,  0.23824902,  0.09606965,  0.37925171,
       -0.19418842, -0.53163167, -0.73460126, -0.616465  , -0.42577499,
       -0.35977475, -0.38015283, -0.15256518,  0.06807301,  0.02058772,
       -0.04910975,  0.05166752, -0.02114529,  0.11004652, -0.09892619,
        0.0488022 ,  0.1782911 , -0.01789395,  0.07572887, -0.02516261,
        0.05056573,  0.01486894, -0.13543435, -0.29474023,  0.3410198 ,
        0.7540293 , -0.03506933,  0.1088104 , -0.88720111, -0.86687313,
       -0.85200511, -0.88923433, -0.86958247, -0.85802647,  0.71012671,
       -0.03219033,  0.1334861 ,  0.73109666, -0.05668145,  0.07073354,
       -0.14081096,  0.53352454, -0.78668412, -0.83825682, -0.89432957,
       -0.87739214, -0.87561486, -0.32400185, -0.72147897, -0.41121801,
       -0.54380961,  0.58649762, -0.62894422,  0.671

# TODO

## Compute MEB from MEBs

# Extension of Chan's Algorithm?